In [1]:
# パッケージのimport
import torch
import torch.nn as nn
import torch.nn.functional as F

# フォルダ「utils」にある関数matchを記述したmatch.pyからimport
from utils.match import match

In [2]:
class MultiBoxLoss(nn.Module):
    #SSDの損失関数のクラス
    
    def __init__(self,jaccard_thresh=0.5, neg_pos=3, devide='cpu'):
        super(MultiBoxLoss,self).__init__()
        self.jaccard_thresh=jaccard_thresh #0.5 関数matchのjaccard係数の閾値
        self.negpos_ratio=neg_pos #3:1 HardNegativeMiningの負と正の比率(背景クラスのデータを全て学習に使うわけではない)
        self.device=device
        
    def forward(self, predictions, targets):
        """
        損失関数の計算
        
        parameters
        predictions:SSD netの訓練時の出力(tuple)
            (loc=torch.Size([num_batch, 8732, 4]), conf=torch.Size([num_batch, 8732, 21]), dbox_list=torch.Size([8732,4]))
        targets:[num_batch, num_objs, 5] 5は正解のアノテーション情報[xmin, ymin, xmax, ymax, label_ind]
        
        returns
        loss_l:tensor locの損失値
        loss_c:tensor confの損失値
        """
        
        #SSDモデルの出力がタプルになっているので個々にバラす
        loc_data, conf_data, dbox_list = predictions
        
        #要素数を把握
        num_batch = loc_data.size(0)
        num_dbox = loc_data.size(1)
        num_classes = loc_data.size(2)
        
        #損失の計算に使用するものを格納する変数作成
        #conf_t_label:各DBoxに一番近い正解のBBoxのラベルを格納する
        #loc_t:各DBoxに一番近い正解のBBoxの位置情報を格納させる
        conf_t_label = torch.LongTensor(num_batch, num_dbox).to(self.device)
        loc_t = torch.Tensor(num_batch, num_dbox, 4).to(self.device)
        
        #loc_tとconf_t_labelに，DBoxと正解アノテーションtargetsをmatchさせた結果を上書きする
        for idx in range(num_batch):
            
            #現在のミニバッチの正解アノテーションのBBoxとラベルを取得
            truths = targets[idx][:,:-1].to(self.device) #BBox
            #ラベル[物体1のラベル，物体2のラベル,...]
            labels = targets[idx][:,-1].to(self.device)
            
            #デフォルトボックスを新たな変数で用意
            dbox = dbox_list.to(self.device)
            
            #関数matchを実行し，loc_t,conf_t_labelの内容を更新する
            #(詳細)参考書p106のあたり
            variance=[0.1,0.2]
            #このvarianceはDBoxからBBoxに補正計算する際に使用する式の係数
            match(self.jaccard_thresh, truths, dbox, variance, labels, loc_t, conf_t_label, idx)
            
        #位置損失:loss_lを計算--------------------
        #smoothL1関数を用いる．ただし，物体を発見したDBoxのオフセットのみを用いる
        #物体を検出したBBoxを取り出すマスクを作成
        pos_mask = conf_t_label > 0
        
        #pos_maskをloc_dataのサイズに変形
        pos_idx = pos_mask.unsqueeze(pos_mask.dim()).expand_as(loc_data)
        
        #Positive DBoxのloc_dataと，教師データloc_tを取得
        loc_p = loc_data[pos_idx].view(-1,4)
        loc_t = loc_t[pos_idx].view(-1,4)
        
        #物体を発見したpositive dboxのオフセット情報loc_tの損失(誤差)を計算
        loss_l = F.smooth_l1_loss(loc_p, loc_t, reduction='sum')
        
        #クラス予測の損失:loss_cを計算---------------------------
        #交差エントロピー誤差関数で損失を計算する．ただし，背景クラスが正解であるDBoxが圧倒的に多いのでHard Negative Miningで
        #物体発見dboxと背景クラスdboxの比が1:3になるようにする
        #そこで背景クラスdboxと予測したもののうち，損失が小さいものはクラス予測の損失から除く
        batch_conf = conf_data.view(-1,4)
        
        #クラス予測の損失を計算(reduction='none'にして，和を取らず，次元を潰さない)
        loss_c = F.cross_entropy(batch_conf, conf_t_label.view(-1), reduction='none')
        
        #これからnegative dboxのうちHard Negative Miningで抽出するものを求めるマスクを作成する
        
        #物体発見したPositive DBoxの損失を0にする．
        num_pos = pos_mask.long().sum(1, keepdim=True) #ミニバッチごとの物体クラス予測の数
        loss_c = loss_c.view(num_batch, -1)
        loss_c[pos_mask] = 0
        
        #Hard Negative Mining. 各DBoxの損失の大きさloss_cの順位であるidx_rankを求める
        _,loss_idx = loss_c.sort(1, descending=True)
        _,idx_rank = loss_idx.sort(1)
        
        #背景のDBoxの数num_negを決める．
        num_neg = torch.clamp(num_pos*self.negpos_ratio, max=num_dbox)
        
        #idx_rankは各DBoxの損失の大きさが上から何番目なのかが入っている
        #背景のDBoxの数num_negより，順位が低い(すなわち損失が多い)DBoxを取るマスク
        neg_mask = idx_rank < (num_neg).expand_as(idx_rank)
        
        #終了
        
        #マスクの形を整形し，conf_dataに合わせる
        #pos_mask:torch.Size([num_batch, 8732])→pos_idx_mask:torch.Size([num_batch, 8732, 21])
        pos_idx_mask = pos_mask.unsqueeze(2).expand_as(conf_data)
        neg_idx_mask = neg_mask.unsqueeze(2).expand_as(conf_data)
        
        #conf_hnm.size([num_pos+num_neg, 21])
        conf_hnm = conf_data[(pos_idx_mask+neg_idx_mask).gt(0)].view(-1, num_classes)
        
        #同様に教師データであるconf_t_labelからpos,negだけを取り出し，conf_label_hnm
        conf_t_label_hnm = conf_t_label[(pos_mask+neg_mask).gt(0)]
        
        #confidenceの損失を計算
        loss_c = F.cross_entropy(conf_hnm, conf_t_label_hnm, reduction='sum')
        
        #物体を発見したBBoxの数N(全ミニバッチの合計)で損失を割る
        N = num_pos.sum()
        loss_l /= N
        loss_c /= N
        
        return loss_l, loss_c